In [1]:
import numpy as np

In [2]:
a = np.arange(6).reshape(3,2).astype(np.float32)
print(a)

[[0. 1.]
 [2. 3.]
 [4. 5.]]


In [3]:
b = np.arange(6).reshape(2,3).astype(np.float32)
print(b)

[[0. 1. 2.]
 [3. 4. 5.]]


In [4]:
matmul_np_golden = np.matmul(a,b)
print(matmul_np_golden)

[[ 3.  4.  5.]
 [ 9. 14. 19.]
 [15. 24. 33.]]


In [ ]:
# tile_k: k维度拆分，weight 和 feature map 都要拆分
# 拆分后 weight和input 相同 tile_id 做matmul，之后所有结果做elemwise add，
# 假设k拆了 tile_k 次，最后有 tile_k 个tensor做加法
#  一般m叫batch方向，k是ic方向，n是oc方向
# 【m,k】* 【k,n】= 【m,n】
# 【n,ic】* 【ic, oc】 = 【n, oc】

In [5]:
m = 3
k = 2
n = 3

tile_k = 2
k_stride = k // tile_k

In [6]:
# 1、feature map 拆 tile_k=2 次
feature_map_tile_k_0 = a[:, :1]
print(feature_map_tile_k_0)

[[0.]
 [2.]
 [4.]]


In [7]:
feature_map_tile_k_1 = a[:, 1:2]
print(feature_map_tile_k_1)

[[1.]
 [3.]
 [5.]]


In [8]:
# 2、weight 拆 tile_k=2 次
weight_tile_k_0 = b[:1, :]
print(weight_tile_k_0)

[[0. 1. 2.]]


In [10]:
weight_tile_k_1 = b[1:2, :]
print(weight_tile_k_1)

[[3. 4. 5.]]


In [21]:
'''
Author: sunflower-you m13119160579@163.com
Date: 2024-03-25 21:04:52
LastEditors: sunflower-you m13119160579@163.com
LastEditTime: 2024-03-26 21:07:15
FilePath: /compiler_server/python/manual_optimization/matmul/tile_k.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
accum_add = None
for tile_k_id in range(tile_k):
    feature_map_tile_k = a[:, tile_k_id*k_stride:(tile_k_id+1)*k_stride]
    weight_tile_k = b[tile_k_id*k_stride:(tile_k_id+1)*k_stride, :]
    matmul_tile_k = np.matmul(feature_map_tile_k, weight_tile_k)
    # print(feature_map_tile_k, weight_tile_k)
    if tile_k_id == 0:
        accum_add = matmul_tile_k
        continue
    accum_add += matmul_tile_k
print(accum_add)

[[ 3.  4.  5.]
 [ 9. 14. 19.]
 [15. 24. 33.]]


In [22]:
print(f"matmul k维度拆成 {tile_k} 分，结果是否一致：", (matmul_np_golden == accum_add).all())

matmul k维度拆成 2 分，结果是否一致： True
